<a href="https://colab.research.google.com/github/bijular/datascience/blob/master/Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Word2vec word embeddings

In [ ]:
from gensim import models
# import the pre-trained model of Google News 6B words
model = models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

# predifined functions from the gensim lib
print(model.most_similar(positive=['woman', 'Doctor'], negative=['man']))

print(model.doesnt_match("spring autumn winter tennis".split()))

print(model.similarity('woman', 'man'))

#GloVe word embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-02-15 16:35:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-02-15 16:35:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-02-15 16:35:20--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 100)

In [ ]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

print(model.most_similar(positive=['woman', 'doctor'], negative=['man']))

print(model.doesnt_match("spring autumn winter tennis".split()))

print(model.similarity('woman', 'man'))

[('nurse', 0.7735227346420288), ('physician', 0.7189429998397827), ('doctors', 0.6824328303337097), ('patient', 0.6750682592391968), ('dentist', 0.6726033687591553), ('pregnant', 0.6642460227012634), ('medical', 0.6520450115203857), ('nursing', 0.645348072052002), ('mother', 0.6393327116966248), ('hospital', 0.6387495994567871)]
tennis
0.8323495


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


#FastText word embeddings

In [ ]:
from gensim.models import FastText
from gensim.test.utils import common_texts
model = FastText( window=3, min_count=1)
model.build_vocab(sentences=common_texts)
model.train(sentences=common_texts, total_examples=len(common_texts), epochs=10)

In [ ]:
similarities = model.wv.most_similar(positive=['computer', 'human'], negative=['interface'])
print(similarities[0])

print(model.wv.doesnt_match("human computer interface tree".split()))

print(model.wv.similarity('computer', 'human'))

('response', 0.09965892136096954)
tree
-0.046674214


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


#LaBSE Sentence Embeddings

In [ ]:
!pip install bert-for-tf2

In [45]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

def get_model(model_url, max_seq_length):
  labse_layer = hub.KerasLayer(model_url, trainable=True)
  # Define input.
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="segment_ids") 

  # LaBSE layer.
  pooled_output,_ = labse_layer([input_word_ids, input_mask, segment_ids])

  # The embedding is l2 normalized.
  pooled_output = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(pooled_output)

  # Define model.
  
  return tf.keras.Model(inputs=[input_word_ids, input_mask, segment_ids],outputs=pooled_output), labse_layer

max_seq_length = 64
labse_model, labse_layer = get_model(model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=max_seq_length)



In [46]:

import bert
max_seq_length=64
labse_layer = hub.KerasLayer("https://tfhub.dev/google/LaBSE/1", trainable=True)
vocab_file = labse_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = labse_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def create_input(input_strings, tokenizer, max_seq_length):
  input_ids_all, input_mask_all, segment_ids_all = [], [], []
  for input_string in input_strings:
    # Tokenize input.
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)


    # Padding or truncation.
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))
    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)
    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    segment_ids_all.append([0] * max_seq_length) 
  return np.array(input_ids_all), np.array(input_mask_all), np.array(segment_ids_all)

def encode(input_text):
  input_ids, input_mask, segment_ids = create_input(
      input_text, tokenizer, max_seq_length)
  return labse_model([input_ids, input_mask, segment_ids])

english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

english_embeddings = encode(english_sentences)
italian_embeddings = encode(italian_sentences)
japanese_embeddings = encode(japanese_sentences)

# English-Italian similarity
print (np.matmul(english_embeddings, np.transpose(italian_embeddings)))

# English-Japanese similarity
print (np.matmul(english_embeddings, np.transpose(japanese_embeddings)))

# Italian-Japanese similarity
print (np.matmul(italian_embeddings, np.transpose(japanese_embeddings)))


[[0.63192725 0.30619976 0.44297647]
 [0.11652887 0.8596669  0.35940808]
 [0.14804009 0.3244818  0.9542653 ]]
[[0.93567264 0.5403088  0.46792448]
 [0.3180447  0.7622262  0.36086115]
 [0.36750826 0.42791754 0.8171463 ]]
[[0.534371   0.25018716 0.19974771]
 [0.30141014 0.7133327  0.4064769 ]
 [0.38503203 0.47768122 0.8674307 ]]


In [47]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 4.9MB/s 
     |████████████████████████████████| 1.8MB 10.5MB/s 
     |████████████████████████████████| 1.2MB 35.1MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
     |████████████████████████████████| 3.2MB 54.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=e1f4a58648f72d99f1848f925a67635c782b92a615581595b60d15c97d9136e7
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=03760dd8810459596cc569f5291fa6f3b7340cb28149148eee28bef3d164c681
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
